In [1]:
import os 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
import shutil 

from tools import train_model


In [2]:

print("---")
print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", PIL.__version__)
print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

print("---")

---
matplotlib version: 3.9.2
pandas version: 2.3.1
PIL version: 10.4.0
torch version: 2.8.0+cpu
torchvision version: 0.23.0+cpu
---


In [3]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cpu device.


In [4]:
class convertToRGB:

    def __call__(self,image):
      if image.mode != "RGB":
        image=image.convert("RGB")
      return image

In [5]:
train_dir=os.path.join("data","train_features")
test_dir=os.path.join('data','train_features')

df=pd.read_csv('data/train_labels.csv')

In [6]:
categories = [
    'monkey_prosimian',
    'antelope_duiker',
    'civet_genet',
    'leopard',
    'blank',
    'rodent',
    'bird',
    'hog'
]

In [7]:
for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)

def org_images(image_dir,df,categories):
  moved_count=0
  skipped_count=0

  for image_file in os.listdir(image_dir):
    image_path=os.path.join(image_dir,image_file)
    if os.path.isdir(image_path):
       continue

    image_name=os.path.splitext(image_file)[0]

    if image_name not in df['id'].values:
      skipped_count +=1
      continue

    image_row=df[df['id']==image_name].iloc[0]
    category_found=None


    for category in categories:
      if image_row[category]==1.0:
        category_found=category
        break

    if category_found:
      dest_path=os.path.join(train_dir,category_found,image_file)
      shutil.move(image_path,dest_path)
      moved_count+=1
    else:
      skipped_count += 1

  return moved_count,skipped_count

In [6]:
moved_count,skipped_count=org_images(train_dir,df,categories)
print(f"moved count :{moved_count}")
print(f"skpped count:{skipped_count}")


NameError: name 'org_images' is not defined

In [8]:
transform=transforms.Compose(
    [
        convertToRGB(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        
    ]
)

In [9]:
train_dataset=datasets.ImageFolder(train_dir,transform=transform)

In [10]:
dataset_loader=DataLoader(train_dataset,batch_size=32)

In [11]:
def get_mean_std(dataloader):
    channel_sum,channels_squared_sum,num_batches=0,0,0
    for data,_ in dataloader:
        channel_sum +=torch.mean(data,dim=[0,2,3])
        channels_squared_sum += torch.mean(data ** 2,dim=[0,2,3])
        num_batches +=1
    mean=channel_sum/num_batches
    std=(channels_squared_sum/num_batches - mean**2)**0.5
    return mean,std

In [12]:
mean,std=get_mean_std(dataset_loader)

In [13]:
transform_norm=transforms.Compose(
    [
        convertToRGB(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean,std=std)
        
    ]
)

In [14]:
train_dataset=datasets.ImageFolder(train_dir,transform=transform_norm)
train_loader=DataLoader(train_dataset,batch_size=32)

## the CNN  vgg6 like model

In [15]:
g=torch.Generator()
g.manual_seed(42)
train_dataset,val_dataset=random_split(train_dataset,[0.8,0.2])


In [16]:

train_loader = DataLoader(train_dataset,batch_size=32)

val_loader = DataLoader(val_dataset,batch_size=32)

single_batch = next(iter(train_loader))[0]
print(f"Shape of one batch: {single_batch.shape}")

Shape of one batch: torch.Size([32, 3, 224, 224])


## the model

In [17]:
class CNNmodel(nn.Module):
    def __init__(self,num_classes):
        super(CNNmodel,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),  
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )


    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



In [18]:
model=CNNmodel(num_classes=len(categories))
print(model)

CNNmodel(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Line

## Training the model

In [19]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)


In [21]:
loss,accu=train_model(model,train_loader,criterion,optimizer,30)

KeyboardInterrupt: 